# Federated Protein Embeddings and Task Model Fitting with BioNeMo

<div class="alert alert-block alert-info"> <b>NOTE</b> This notebook was tested on a single A1000 GPU and is compatible with BioNeMo Framework v2.2. To leverage additional or higher-performance GPUs, you can modify the configuration files and simulation script to accommodate multiple devices and increase thread utilization respectively. </div>

This example notebook shows how to obtain protein learned representations in the form of embeddings using the ESM-1nv pre-trained model in a federated learning (FL) setting. The model is trained with NVIDIA's BioNeMo framework for Large Language Model training and inference. For more details, please visit NVIDIA BioNeMo Service at https://www.nvidia.com/en-us/gpu-cloud/bionemo.

This example is based on NVIDIA BioNeMo Service [example](https://github.com/NVIDIA/BioNeMo/blob/main/examples/service/notebooks/task-fitting-predictor.ipynb) 
but runs inference locally (on the FL clients) instead of using BioNeMo's cloud API.

This notebook will walk you through the task fitting workflow in the following sections:

* Dataset sourcing & Data splitting
* Federated embedding extraction
* Training an MLP to predict subcellular location

## Setup

Ensure that you have read through the Getting Started section, can run the BioNeMo Framework docker container, and have configured the NGC Command Line Interface (CLI) within the container. It is assumed that this notebook is being executed from within the container.

<div class="alert alert-block alert-info"> <b>NOTE</b> Some of the cells below generate long text output.  We're using <pre>%%capture --no-display --no-stderr cell_output</pre> to suppress this output.  Comment or delete this line in the cells below to restore full output.</div>

### Import and install all required packages

In [1]:
#%%capture --no-display --no-stderr cell_output
! pip install nvflare~=2.5
! pip install biopython --no-dependencies
#! pip install scikit-learn
#! pip install matplotlib

import io
import numpy as np
import os
import pickle
import re
import requests
import split_data

from Bio import SeqIO
from nvflare import SimulatorRunner  
from split_data import split

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/opt_einsum-3.4.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/dill-0.3.9-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enf

### Obtaining the protein embeddings using the BioNeMo ESM-1nv model
Using BioNeMo, each FL client can obtain numerical vector representations of protein sequences called embeddings. Protein embeddings can then be used for visualization or making downstream predictions.

Here we are interested in training a neural network to predict subcellular location from an embedding.

The data we will be using comes from the paper [Light attention predicts protein location from the language of life](https://academic.oup.com/bioinformaticsadvances/article/1/1/vbab035/6432029) by Stärk et al. In this paper, the authors developed a machine learning algorithm to predict the subcellular location of proteins from sequence through protein langage models that are similar to those hosted by BioNeMo. Protein subcellular location refers to where the protein localizes in the cell, for example a protein my be expressed in the Nucleus or in the Cytoplasm. Knowing where proteins localize can provide insights into the underlying mechanisms of cellular processes and help identify potential targets for drug development. The following image includes a few examples of subcellular locations in an animal cell:


(Image freely available at https://pixabay.com/images/id-48542)

### Dataset sourcing
For our target input sequences, we will point to FASTA sequences in a benchmark dataset called Fitness Landscape Inference for Proteins (FLIP). FLIP encompasses experimental data across adeno-associated virus stability for gene therapy, protein domain B1 stability and immunoglobulin binding, and thermostability from multiple protein families.

In [2]:
# Example protein dataset location
fasta_url= "http://data.bioembeddings.com/public/FLIP/fasta/scl/mixed_soft.fasta"

First, we define the source of example protein dataset with the FASTA sequences. This data follows the [biotrainer](https://github.com/sacdallago/biotrainer/blob/main/docs/data_standardization.md) standard, so it includes information about the class in the FASTA header, and the protein sequence. Here are two example sequences in this file:

```
>Sequence1 TARGET=Cell_membrane SET=train VALIDATION=False
MMKTLSSGNCTLNVPAKNSYRMVVLGASRVGKSSIVSRFLNGRFEDQYTPTIEDFHRKVYNIHGDMYQLDILDTSGNHPFPAM
RRLSILTGDVFILVFSLDSRESFDEVKRLQKQILEVKSCLKNKTKEAAELPMVICGNKNDHSELCRQVPAMEAELLVSGDENC
AYFEVSAKKNTNVNEMFYVLFSMAKLPHEMSPALHHKISVQYGDAFHPRPFCMRRTKVAGAYGMVSPFARRPSVNSDLKYIKA
KVLREGQARERDKCSIQ
>Sequence4833 TARGET=Nucleus SET=train VALIDATION=False
MARTKQTARKSTGGKAPRKQLATKAARKSAPATGGVKKPHRFRPGTVALREIRKYQKSTELLIRKLPFQRLVREIAQDFKTDL
RFQSSAVAALQEAAEAYLVGLFEDTNLCAIHAKRVTIMPKDIQLARRIRGERA
Note the following attributes in the FASTA header:
```

* `TARGET` attribute holds the subcellular location classification for the sequence, for instance Cell_membrane and Nucleus. This dataset includes a total of ten subcellelular location classes -- more on that below.
* `SET` attribute defines whether the sequence should be used for training (train) or testing (test)
* `VALIDATION` attribute defines whether the sequence should be used for validation (all sequences where this is True are also in set=train)

### Downloading the protein sequences and subcellular location annotations
In this step we download the FASTA file defined above and parse the sequences into a list of BioPython SeqRecord objects.



In [3]:
# Download the FASTA file from FLIP: https://github.com/J-SNACKKB/FLIP/tree/main/splits/scl
fasta_content = requests.get(fasta_url, headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x86)'
}).content.decode('utf-8')
fasta_stream = io.StringIO(fasta_content)

# Obtain a list of SeqRecords/proteins which contain sequence and attributes
# from the FASTA header
proteins = list(SeqIO.parse(fasta_stream, "fasta"))
print(f"Downloaded {len(proteins)} sequences")

Downloaded 13949 sequences


### Download Model Checkpoints

The following code will download the pre-trained model, `esm2n/650m:2.0`, from the NGC registry:

In [4]:
from bionemo.core.data.load import load

checkpoint_path = load("esm2/8m:2.0")
print(checkpoint_path)

{
    "download_end": "2025-02-13 01:17:56",
    "download_start": "2025-02-13 01:17:55",
    "download_time": "1s",
    "files_downloaded": 1,
    "local_path": "/root/.cache/bionemo/tmp5hpuy5_0/esm2nv8m_v2.0",
    "size_downloaded": "16.97 MB",
    "status": "COMPLETED"
}


Untarring contents of '/root/.cache/bionemo/2957b2c36d5978d0f595d6f1b72104b312621cf0329209086537b613c1c96d16-esm2_hf_converted_8m_checkpoint.tar.gz' to '/root/.cache/bionemo/2957b2c36d5978d0f595d6f1b72104b312621cf0329209086537b613c1c96d16-esm2_hf_converted_8m_checkpoint.tar.gz.untar'


/root/.cache/bionemo/2957b2c36d5978d0f595d6f1b72104b312621cf0329209086537b613c1c96d16-esm2_hf_converted_8m_checkpoint.tar.gz.untar


### Data splitting
Next, we prepare the data for simulating federated learning using `n_clients`. Note that a copy of the same test set is shared between the clients in this example.

In [5]:
n_clients = 3
# limiting to the proteins with sequence length<512 for embedding queries
MAX_SEQUENCE_LEN = 512
seed=0
data_root = "/tmp/data/mixed_soft"
split_alpha = 1.0  # moderate label heterogeneity of alpha=1.0

np.random.seed(seed)

# Extract meta data and split
data = []
for i, x in enumerate(proteins):
        if len(str(x.seq)) > MAX_SEQUENCE_LEN:
            continue
            
        entry = {key: value for key, value in re.findall(r"([A-Z_]+)=(-?[A-z0-9]+[.0-9]*)", x.description)}
        entry["sequences"] = str(x.seq)
        entry["id"] = str(i)
        entry["labels"] = entry["TARGET"]
       
        data.append(entry)
print(f"Read {len(data)} valid sequences.")
               
# Split the data and save for each client
# Note, test_data is kept the same on each client and is not split
# `concat=False` is used for SCL experiments (see ../downstream/scl)
split(proteins=data, num_sites=n_clients, split_dir=data_root, alpha=split_alpha, concat=False)  
# `concat=True` is used for separate inference + MLP classifier in this notebook
split(proteins=data, num_sites=n_clients, split_dir=data_root, alpha=split_alpha, concat=True)  

Read 8619 valid sequences.
Partition protein dataset with 10 classes into 3 sites with Dirichlet sampling under alpha 1.0
{'site-1': {'Cell_membrane': 137,
            'Cytoplasm': 311,
            'Endoplasmic_reticulum': 123,
            'Extracellular': 356,
            'Golgi_apparatus': 43,
            'Lysosome': 39,
            'Mitochondrion': 250,
            'Nucleus': 453,
            'Peroxisome': 23,
            'Plastid': 110},
 'site-2': {'Cell_membrane': 217,
            'Cytoplasm': 491,
            'Endoplasmic_reticulum': 195,
            'Extracellular': 562,
            'Golgi_apparatus': 70,
            'Lysosome': 63,
            'Mitochondrion': 396,
            'Nucleus': 716,
            'Peroxisome': 38,
            'Plastid': 174},
 'site-3': {'Cell_membrane': 160,
            'Cytoplasm': 361,
            'Endoplasmic_reticulum': 144,
            'Extracellular': 414,
            'Golgi_apparatus': 51,
            'Lysosome': 47,
            'Mitochondrion'

### Federated embedding extraction
Here, we run inference on each client sequentially using one thread to preserve GPU memory. Running inference of the ESM-1nv model to extract embeddings with a micro-batch size of 64 takes about 30 GB GPU memory. You can reduce the micro-batch size if you have less GPU memory available. For more inference requirements, see the [model overview](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/clara/models/esm2nv650m). Note that this can take a while.

In [ ]:
results_path = "/tmp/data/mixed_soft/results"

micro_bs = 64

if not os.path.isdir(results_path):
    os.makedirs(results_path)

import sys
sys.path.append(os.path.join(os.getcwd(), "src"))

from bionemo_inference import BioNeMoInference
from bionemo_launcher import BioNeMoLauncher

from nvflare.app_opt.pt.job_config.base_fed_job import BaseFedJob
from nvflare.app_common.launchers.subprocess_launcher import SubprocessLauncher

# Create BaseFedJob
job = BaseFedJob(
  name="esm2_embeddings"
)

# Define the controller and send to server
controller = BioNeMoInference(
    min_responses_required=n_clients
)
job.to_server(controller)

# Add clients
for i in range(n_clients):
    # build inference command for each client
    client_name = f"site-{i+1}"
    command = f"infer_esm2 --checkpoint-path {checkpoint_path} --data-path {data_root}/data_{client_name}.csv --results-path {results_path}/inference_results_{client_name} --precision bf16-mixed --include-embeddings --include-logits --include-input-ids --micro-batch-size {micro_bs} --num-gpus 1"
    
    launcher = BioNeMoLauncher(
        launcher_id=job.as_id(SubprocessLauncher(script=command, launch_once=True))
    )
    job.to(launcher, client_name)

job.export_job(f"./exported_jobs/embeddings")  # optional export of job configurations. Can be used to submit the job to a real-world NVFlare deployment.
job.simulator_run("/tmp/nvflare/bionemo/embeddings", gpu="0", threads=1)  # due to memory constraints, we run the client's inference sequentially in one thread and gpu.

### Inspecting the embeddings and labels
Embeddings returned from the BioNeMo model are vectors of fixed size for each input sequence. In other words, if we input 10 sequences, we will obtain a matrix `10xD`, where `D` is the size of the embedding (in the case of ESM2-650m, `D=1280`). At a glance, these real-valued vector embeddings don't show any obvious features (see the printout in the next cell). But these vectors do contain information that can be used in downstream models to reveal properties of the protein, for example the subcellular location as we'll explore below.

In [ ]:
# load embeddings from site-1
import torch

client_name = "site-1"
results = torch.load(f"{results_path}/inference_results_{client_name}/predictions__rank_0.pt")

for key, val in results.items():
    if val is not None:
        print(f'{key}\t{val.shape}')

protein_embeddings = results['embeddings']
print(f"Loaded {len(protein_embeddings)} embeddings from site-1.")

for i in range(4):
    x = protein_embeddings[i]
    print(f"{i+1}. embedding: range {x.min():.2f}-{x.max():.2f}, mean={x.mean():.2f}, shape={x.shape}")

Let's enumerate the labels corresponding to potential subcellular locations.

In [ ]:
# Let's also print all the labels

labels = set([entry['labels'] for entry in data])

for i, label in enumerate(labels):
    print(f"{i+1}. {label.replace('_', ' ')}")

### Training a MLP to predict subcellular location
To be able to classify proteins for their subcellular location, we train a simple scikit-learn Multi-layer Perceptron (MPL) classifier using Federated Averaging ([FedAvg](https://arxiv.org/abs/1602.05629)). The MLP model uses a network of hidden layers to fit the input embedding vectors to the model classes (the cellular locations above). In the simulation below, we define the MLP to use the Adam optimizer with a network of (512, 256, 128) hidden layers, defining a random state (or seed) for reproducibility, and trained for 30 rounds of FedAvg (see [config_fed_server.json](./jobs/fedavg/app/config/config_fed_server.json)). 

We can use the same configuration also to simulate local training where each client is only training with their own data by setting `os.environ["SIM_LOCAL"] = "True"`. Our [BioNeMoMLPLearner](./jobs/fedavg/app/custom/bionemo_mlp_learner.py) will then ignore the global weights coming from the server.

### Local training

In [ ]:
os.environ["SIM_LOCAL"] = "True"

from bionemo_mlp_learner import BioNeMoMLPLearner
from bionemo_mlp_job import BioNeMoMLPJob

from nvflare.app_common.workflows.fedavg import FedAvg
from nvflare.app_common.executors.model_learner_executor import ModelLearnerExecutor

# Create BioNeMoMLPJob which uses a custom model persistor for the sklearn MLP model
job = BioNeMoMLPJob(
  name="mlp_local"
)

# Define the controller and send it to server
controller = FedAvg(
    num_clients=n_clients,
    num_rounds=100,
)
job.to_server(controller)

# Add clients
for i in range(n_clients):
    client_name = f"site-{i+1}"
    executor = ModelLearnerExecutor(learner_id=job.as_id(
            BioNeMoMLPLearner(
                data_path=os.path.join(data_root, f"data_{client_name}.csv"),
                inference_result=f"{results_path}/inference_results_{client_name}/predictions__rank_0.pt",
                aggregation_epochs=4,
                lr=1e-5                
            )
    ))
    job.to(executor, client_name)    

job.export_job(f"./exported_jobs/local_alpha{split_alpha}")  # optional export of job configurations. Can be used to submit the job to a real-world NVFlare deployment.
job.simulator_run(f"/tmp/nvflare/bionemo/local_alpha{split_alpha}", threads=n_clients)  # As the MLP is very lightweight, we can run all clients in parallel on the CPU.

### Federated learning

In [ ]:
os.environ["SIM_LOCAL"] = "False"

from bionemo_mlp_learner import BioNeMoMLPLearner
from bionemo_mlp_job import BioNeMoMLPJob

from nvflare.app_common.workflows.fedavg import FedAvg
from nvflare.app_common.executors.model_learner_executor import ModelLearnerExecutor

# Create BioNeMoMLPJob which uses a custom model persistor for the sklearn MLP model
job = BioNeMoMLPJob(
  name="mlp_fedavg"
)

# Define the controller and send it to server
controller = FedAvg(
    num_clients=n_clients,
    num_rounds=100,
)
job.to_server(controller)

# Add clients
for i in range(n_clients):
    client_name = f"site-{i+1}"
    executor = ModelLearnerExecutor(learner_id=job.as_id(
            BioNeMoMLPLearner(
                data_path=os.path.join(data_root, f"data_{client_name}.csv"),
                inference_result=f"{results_path}/inference_results_{client_name}/predictions__rank_0.pt",
                aggregation_epochs=4,
                lr=1e-5                
            )
    ))
    job.to(executor, client_name)    

job.export_job(f"./exported_jobs/fedavg_alpha{split_alpha}")  # optional export of job configurations. Can be used to submit the job to a real-world NVFlare deployment.
job.simulator_run(f"/tmp/nvflare/bionemo/fedavg_alpha{split_alpha}", threads=n_clients)  # As the MLP is very lightweight, we can run all clients in parallel on the CPU.

### TensorBoard Visualization
You can visualize the training progress using TensorBoard
```
tensorboard --logdir /tmp/nvflare/bionemo
```

Note, as the test set is shared between the clients in this example, we will see the same performance metrics for the global model computed on each client.

An example of local (green, blue, red) vs. federated (orange) training is shown below.

![TensorBoard training curves](tb_curve.png)